Librerías y Funciones propias:

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('..')) 

import numpy as np
import pyroomacoustics as pra
from scipy.io import wavfile
import pandas as pd
from itertools import combinations
from algoritmos.CC import timeAveragedcrossCorrelation
from algoritmos.GCC import generalizedCrossCorrelation
from algoritmos.DOA import doa
from helpers.addNoise import addNoise

Señales de entrada:

In [14]:

fs, sourceSignal = wavfile.read("../audios/p262_001.wav")
sourceSignal = addNoise(sourceSignal, 30)

if sourceSignal.dtype != np.float32 and sourceSignal.dtype != np.float64:
    sourceSignal = sourceSignal.astype(np.float32)

if np.max(np.abs(sourceSignal)) > 1:
    sourceSignal = sourceSignal / np.max(np.abs(sourceSignal))

Creamos los rooms:

In [15]:
distancesBetweenMicsDf = pd.read_excel("../datos y resultados/Datos.xlsx", engine='openpyxl', header=None)
distancesBetweenMics = distancesBetweenMicsDf.iloc[8, 4:105].to_numpy() / 100

amountsOfMics = [2, 4, 8]

roomsMicSeparation = []

for distanceBetweenMics in distancesBetweenMics:

    # Room dimensions (width * length)
    roomDimensions = [8, 8, 3]

    # Create a room
    rt60Target = 0.5
    eAbsorption, max_order = pra.inverse_sabine(rt60Target, roomDimensions)
    room = pra.ShoeBox(
        roomDimensions, fs=fs, materials=pra.Material(eAbsorption), max_order=max_order
    )

    # Sound source position (x,y,z)
    sourcePosition = [6.828, 5.328, 1.5]

    # Add the source to the room
    room.add_source(sourcePosition, signal=sourceSignal)

    # Define a linear Microphone Array: 4 mics in a row
    initialX = 4 - (3 * distanceBetweenMics) / 2
    initialMicPosition = [initialX, 2.5, 1]

    # Calculate positions for 4 microphones
    micPositions = np.array([
        [initialMicPosition[0] + i * distanceBetweenMics,
        initialMicPosition[1],
        initialMicPosition[2]]
        for i in range(4)
    ])

    # Add microphones to the room
    room.add_microphone_array(pra.MicrophoneArray(micPositions.T, room.fs))

    roomsMicSeparation.append(room)


roomsAmountOfMics = []

for amountOfMics in amountsOfMics:

    # Room dimensions (width * length)
    roomDimensions = [8, 8, 3]

    # Create a room
    rt60Target = 0.5
    eAbsorption, max_order = pra.inverse_sabine(rt60Target, roomDimensions)
    room = pra.ShoeBox(
        roomDimensions, fs=48000, materials=pra.Material(eAbsorption), max_order=max_order
    )

    # Sound source position (x,y,z)
    sourcePosition = [6.828, 5.328, 1.5]

    # Add the source to the room
    room.add_source(sourcePosition, signal=sourceSignal)

    # Define a linear Microphone Array: n mics in a row
    initialX = 4 - ((amountOfMics - 1) * 0.15) / 2
    initialMicPosition = [initialX, 2.5, 1]

    # Calculate positions for 4 microphones
    micPositions = np.array([
        [initialMicPosition[0] + i * 0.15, 
        initialMicPosition[1],
        initialMicPosition[2]]
        for i in range(amountOfMics)
    ])

    # Add microphones to the room
    room.add_microphone_array(pra.MicrophoneArray(micPositions.T, room.fs))

    roomsAmountOfMics.append(room)

Simulamos los rooms

In [16]:

micSignalsArraysMicSeparation = []

for room in roomsMicSeparation:
    # Run the room simulation
    room.simulate()

    # Get the microphone signals
    micSignals = room.mic_array.signals
    micSignalsArraysMicSeparation.append(micSignals)

micSignalsArraysAmountOfMics = []

for room in roomsAmountOfMics:
    # Run the room simulation
    room.simulate()

    # Get the microphone signals
    micSignals = room.mic_array.signals
    micSignalsArraysAmountOfMics.append(micSignals)

# Revisar un room simulado
room = roomsMicSeparation[0]

print("Sample rate:", room.fs)
print("Duración de la señal (s):", len(room.sources[0].signal) / room.fs)
print("Forma de micSignals:", micSignalsArraysMicSeparation[0].shape)

for i in range(4):
    mic = micSignalsArraysMicSeparation[0][i]
    print(f"Mic {i} - max: {np.max(np.abs(mic))}, min: {np.min(mic)}")

Sample rate: 48000
Duración de la señal (s): 2.348520833333333
Forma de micSignals: (4, 181510)
Mic 0 - max: 0.5178148057446341, min: -0.5178148057446341
Mic 1 - max: 0.514308293289882, min: -0.5136104066447112
Mic 2 - max: 0.6517257146327097, min: -0.6517257146327097
Mic 3 - max: 0.5340695657365778, min: -0.5340695657365778


Algoritmos

Time Averaged Cross Correlation

In [ ]:
delays = []
for i in range(3):
    delays.append((timeAveragedcrossCorrelation(micSignals[i], micSignals[i + 1], returnDelay=True, graphs=False)[1]) / fs)

print("Delays between microphones (in seconds):", delays)

print("DOA time averaged cross correlation: ", doa(delays, distanceBetweenMics))

GCC: Classic

In [ ]:
doaClassicMicSeparation = []

# Obtain an array of distances between pair of microphones for each distanceBetweenMics in distancesBetweenMics
distancesBetweenMicsMicSeparation = []
for distanceBetweenMics in distancesBetweenMics:
    distancesBetweenMicsMicSeparation.append([distanceBetweenMics, 2*distanceBetweenMics, 3*distanceBetweenMics, distanceBetweenMics, 2*distanceBetweenMics, distanceBetweenMics])

for micSignals, distanceBetweenMics in zip(micSignalsArraysMicSeparation, distancesBetweenMicsMicSeparation):
    delays = []

    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA classic cross correlation: ", doa(delays, distanceBetweenMics))

    doaClassicMicSeparation.append(doa(delays, distanceBetweenMics))

doaClassicAmountOfMics = []

for micSignals in micSignalsArraysAmountOfMics:
    nMics = micSignals.shape[0]
    pairs = list(combinations(range(nMics), 2))
    print("Pairs of microphones:", pairs)
    
    delays = []
    distances = []
    
    for i, j in pairs:
        delays.append((generalizedCrossCorrelation(micSignals[i], micSignals[j], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
        distances.append((j - i) * 0.15)

    print("Delays between microphones (in seconds):", delays)

    print("DOA classic cross correlation: ", doa(delays, distances))

    doaClassicAmountOfMics.append(doa(delays, distances))


dfMicSeparation = pd.DataFrame(doaClassicMicSeparation, columns=['DOA_Classic_MicSeparation'])
dfAmountOfMics = pd.DataFrame(doaClassicAmountOfMics, columns=['DOA_Classic_AmountOfMics'])

with pd.ExcelWriter('../datos y resultados/doa_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    dfMicSeparation.to_excel(writer, sheet_name='DOA_Classic_MicSeparation', index=False)
    dfAmountOfMics.to_excel(writer, sheet_name='DOA_Classic_AmountOfMics', index=False)

Delays between microphones (in seconds): [8.333333333333333e-05, 8.333333333333333e-05, 0.00020833333333333335, 6.25e-05, 0.00016666666666666666, 8.333333333333333e-05]
DOA classic cross correlation:  60.825611189961954
Delays between microphones (in seconds): [8.333333333333333e-05, 8.333333333333333e-05, 0.0001875, 6.25e-05, 0.00016666666666666666, 0.00010416666666666667]
DOA classic cross correlation:  61.1904348148995
Delays between microphones (in seconds): [0.00010416666666666667, 0.00010416666666666667, 0.00020833333333333335, 4.1666666666666665e-05, 0.0001875, 0.00010416666666666667]
DOA classic cross correlation:  60.58609727009326
Delays between microphones (in seconds): [0.00010416666666666667, 0.00010416666666666667, 0.00020833333333333335, 0.00010416666666666667, 0.00010416666666666667, 0.00010416666666666667]
DOA classic cross correlation:  60.67935777521736
Delays between microphones (in seconds): [0.00010416666666666667, 0.00010416666666666667, 0.00020833333333333335, 0

GCC: Roth

In [18]:
doaRothMicSeparation = []

for micSignals, distanceBetweenMics in zip(micSignalsArraysMicSeparation, distancesBetweenMicsMicSeparation):
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA Roth: ", doa(delays, distanceBetweenMics))

    doaRothMicSeparation.append(doa(delays, distanceBetweenMics))

doaRothAmountOfMics = []

for micSignals in micSignalsArraysAmountOfMics:
    nMics = micSignals.shape[0]
    pairs = list(combinations(range(nMics), 2))
    
    delays = []
    distances = []
    
    for i, j in pairs:
        delays.append(generalizedCrossCorrelation(micSignals[i], micSignals[j], mode='roth', returnDelay=True, graphs=False)[1] / fs)
        distances.append((j - i) * 0.15)

    print("Delays between microphones (in seconds):", delays)

    print("DOA Roth: ", doa(delays, distances))

    doaRothAmountOfMics.append(doa(delays, distances))


dfMicSeparation = pd.DataFrame(doaRothMicSeparation, columns=['DOA_Roth_MicSeparation'])
dfAmountOfMics = pd.DataFrame(doaRothAmountOfMics, columns=['DOA_Roth_AmountOfMics'])

with pd.ExcelWriter('../datos y resultados/doa_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    dfMicSeparation.to_excel(writer, sheet_name='DOA_Roth_MicSeparation', index=False)
    dfAmountOfMics.to_excel(writer, sheet_name='DOA_Roth_AmountOfMics', index=False)

Delays between microphones (in seconds): [8.333333333333333e-05, 0.00010416666666666667, 0.0003125, 4.1666666666666665e-05, 0.00020833333333333335, 0.00010416666666666667]
DOA Roth:  55.12727594499811
Delays between microphones (in seconds): [0.00010416666666666667, 0.00010416666666666667, 0.0003333333333333333, 0.00010416666666666667, 0.00020833333333333335, 0.00010416666666666667]
DOA Roth:  50.33533120584519
Delays between microphones (in seconds): [0.00010416666666666667, 0.00010416666666666667, 0.0003333333333333333, 0.00010416666666666667, 0.00010416666666666667, 0.00010416666666666667]
DOA Roth:  56.11503560109532
Delays between microphones (in seconds): [0.00010416666666666667, 0.0001875, 0.0003541666666666667, 0.00010416666666666667, 0.00010416666666666667, 0.00010416666666666667]
DOA Roth:  54.646451898113376
Delays between microphones (in seconds): [0.000125, 0.00025, 0.0002708333333333333, 0.00010416666666666667, 0.00025, 0.00010416666666666667]
DOA Roth:  49.83467039978618

GCC: SCOT

In [19]:
doaScotMicSeparation = []

for micSignals, distanceBetweenMics in zip(micSignalsArraysMicSeparation, distancesBetweenMicsMicSeparation):
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA SCOT: ", doa(delays, distanceBetweenMics))

    doaScotMicSeparation.append(doa(delays, distanceBetweenMics))

doaScotAmountOfMics = []

for micSignals in micSignalsArraysAmountOfMics:
    nMics = micSignals.shape[0]
    pairs = list(combinations(range(nMics), 2))
    
    delays = []
    distances = []
    
    for i, j in pairs:
        delays.append(generalizedCrossCorrelation(micSignals[i], micSignals[j], mode='scot', returnDelay=True, graphs=False)[1] / fs)
        distances.append((j - i) * 0.15)

    print("Delays between microphones (in seconds):", delays)

    print("DOA SCOT: ", doa(delays, distances))

    doaScotAmountOfMics.append(doa(delays, distances))


dfMicSeparation = pd.DataFrame(doaScotMicSeparation, columns=['DOA_SCOT_MicSeparation'])
dfAmountOfMics = pd.DataFrame(doaScotAmountOfMics, columns=['DOA_SCOT_AmountOfMics'])

with pd.ExcelWriter('../datos y resultados/doa_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    dfMicSeparation.to_excel(writer, sheet_name='DOA_SCOT_MicSeparation', index=False)
    dfAmountOfMics.to_excel(writer, sheet_name='DOA_SCOT_AmountOfMics', index=False)

Delays between microphones (in seconds): [8.333333333333333e-05, 0.00010416666666666667, 0.0003125, 4.1666666666666665e-05, 0.00020833333333333335, 0.00010416666666666667]
DOA SCOT:  55.12727594499811
Delays between microphones (in seconds): [0.00010416666666666667, 0.00010416666666666667, 0.0003333333333333333, 4.1666666666666665e-05, 0.00020833333333333335, 0.00010416666666666667]
DOA SCOT:  54.85041895098342
Delays between microphones (in seconds): [0.00010416666666666667, 0.00010416666666666667, 0.0003333333333333333, 0.00010416666666666667, 0.00010416666666666667, 0.00010416666666666667]
DOA SCOT:  56.11503560109532
Delays between microphones (in seconds): [0.00010416666666666667, 0.0001875, 0.0003541666666666667, 0.00010416666666666667, 0.00010416666666666667, 0.00010416666666666667]
DOA SCOT:  54.646451898113376
Delays between microphones (in seconds): [0.000125, 0.00025, 0.000375, 0.00010416666666666667, 0.00025, 0.00010416666666666667]
DOA SCOT:  47.41152731435643
Delays betwe

GCC: Phase

In [20]:
doaPhaseMicSeparation = []

for micSignals, distanceBetweenMics in zip(micSignalsArraysMicSeparation, distancesBetweenMicsMicSeparation):
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA Phase: ", doa(delays, distanceBetweenMics))

    doaPhaseMicSeparation.append(doa(delays, distanceBetweenMics))

doaPhaseAmountOfMics = []

for micSignals in micSignalsArraysAmountOfMics:
    nMics = micSignals.shape[0]
    pairs = list(combinations(range(nMics), 2))
    
    delays = []
    distances = []
    
    for i, j in pairs:
        delays.append(generalizedCrossCorrelation(micSignals[i], micSignals[j], mode='phase', returnDelay=True, graphs=False)[1] / fs)
        distances.append((j - i) * 0.15)

    print("Delays between microphones (in seconds):", delays)

    print("DOA Phase: ", doa(delays, distances))

    doaPhaseAmountOfMics.append(doa(delays, distances))


dfMicSeparation = pd.DataFrame(doaPhaseMicSeparation, columns=['DOA_Phase_MicSeparation'])
dfAmountOfMics = pd.DataFrame(doaPhaseAmountOfMics, columns=['DOA_Phase_AmountOfMics'])

with pd.ExcelWriter('../datos y resultados/doa_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    dfMicSeparation.to_excel(writer, sheet_name='DOA_Phase_MicSeparation', index=False)
    dfAmountOfMics.to_excel(writer, sheet_name='DOA_Phase_AmountOfMics', index=False)

Delays between microphones (in seconds): [8.333333333333333e-05, 0.00010416666666666667, 0.0003125, 4.1666666666666665e-05, 0.00020833333333333335, 0.00010416666666666667]
DOA Phase:  55.12727594499811
Delays between microphones (in seconds): [0.00010416666666666667, 0.00010416666666666667, 0.0003333333333333333, 4.1666666666666665e-05, 0.00020833333333333335, 0.00010416666666666667]
DOA Phase:  54.85041895098342
Delays between microphones (in seconds): [0.00010416666666666667, 0.00022916666666666666, 0.0003333333333333333, 0.00010416666666666667, 0.00010416666666666667, 0.00010416666666666667]
DOA Phase:  51.67254206678678
Delays between microphones (in seconds): [0.00010416666666666667, 0.0001875, 0.0003541666666666667, 0.00010416666666666667, 0.00010416666666666667, 0.00010416666666666667]
DOA Phase:  54.646451898113376
Delays between microphones (in seconds): [0.000125, 0.00025, 0.000375, 0.00010416666666666667, 0.00025, 0.00010416666666666667]
DOA Phase:  47.41152731435643
Delays 